In [1]:
import dask
import dask.bag as db
import dask.dataframe as dd
from dask.distributed import get_worker
from dask.distributed import Client, progress

In [2]:
# client = Client(processes=True)
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB', processes=True)
client

Client Scheduler: tcp://127.0.0.1:34733 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [3]:
# client.close()

In [4]:
import glob
import spacy
import pandas as pd

from wb_nlp import dir_manager
from wb_nlp.cleaning import cleaner

In [5]:
f = [f for f in glob.glob(dir_manager.get_path_from_root('src', 'wb_nlp', 'cleaning', '*.py'))]

In [10]:
def read_file(f):
    
    worker = get_worker()

    try:
        nlp = worker.nlp
        lda_cleaner = worker.lda_cleaner
    except AttributeError:
        nlp = spacy.load('en_core_web_sm')
        lda_cleaner = cleaner.LDACleaner()
        worker.nlp = nlp
        worker.lda_cleaner = lda_cleaner

    content = {}
    with open(f) as fl:
        tokens = lda_cleaner.get_tokens_and_phrases(fl.read())
        content[f] = tokens
        # doc = nlp(fl.read())
        # content[f] = [token.lemma_ for token in doc]
        with open(f + '.txt', 'w') as ff:
            fl.seek(0)
            ff.write(fl.read())

    # return True

    return content

In [11]:
b = db.from_sequence(f * 10, npartitions=4)
file_reader = b.map(read_file)

In [12]:
%%time
# res = file_reader.all().compute()
res = file_reader.compute()

CPU times: user 876 ms, sys: 1.09 s, total: 1.97 s
Wall time: 14.8 s


In [14]:
client.cancel(b)

In [15]:
client.restart()

distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing


Client Scheduler: tcp://127.0.0.1:34733 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [12]:
%%time
res = file_reader.compute()

CPU times: user 718 ms, sys: 446 ms, total: 1.16 s
Wall time: 5.85 s


In [14]:
%%time
res = file_reader.compute()

CPU times: user 5.37 s, sys: 3.42 s, total: 8.79 s
Wall time: 50.7 s
